In [ ]:
import pdfplumber
import pandas as pd
from pathlib import Path
import re

In [ ]:
files = Path("EstadosDeCuenta")
for file in files.iterdir():
    if file.is_file():
        print(file)
    file_path = file
    words = file.stem.split("_")
    month = words[-1]
    msi = {}
    spend = {}
    
    with pdfplumber.open(file_path) as pdf:
        page = pdf.pages[3]
        setting = {
            "vertical_strategy": "lines",
            "horizontal_strategy": "lines",
            "snap_tolerance": 5, 
            "join_tolerance": 5,
            "intersection_tolerance": 5,
                    
        }
        table = page.extract_tables(table_settings=setting)
        
        if table:
            df1 = pd.DataFrame(table[0][1:], columns=table[0][0])
            df1 = df1.replace([r'\n', 'Tarjeta Digital',r'\*\*\*9552', 'Tarjeta titular: XXXXXXXXXXXX7445'], ' ', regex=True)
            df1.columns.values[0] = month
            msi[month] = df1
            
            df2 = pd.DataFrame(table[1][1:], columns=table[1][0])
            df2 = df2.replace([r'\n', 'Tarjeta Digital', r'\*\*\*9552', 'Tarjeta titular: XXXXXXXXXXXX7445	'], ' ', regex=True)
            df2.columns.values[0] = month
            spend[month] = df1
            
        else:
            print("Tablen't")    
        im = page.to_image(resolution=150)
        im.reset().debug_tablefinder(setting)    
    

    
    
    
   

In [ ]:
msi

In [ ]:
im.debug_tablefinder()
im.reset().debug_tablefinder(setting)